In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math
from matplotlib import pyplot
from matplotlib.image import imread
import tensorflow as tf
from tensorflow import keras
import cv2
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.python.keras.utils.vis_utils import model_to_dot
#from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import glorot_uniform


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

with open('/home/garrett/KagglesData/Butterflies/fgvc_fg_training.json','r') as anno_train:
    train = json.load(anno_train)
with open('/home/garrett/KagglesData/Butterflies/fgvc_fg_testing.json','r') as anno_test:
    test = json.load(anno_test)

#print(train)   


train.keys()
test.keys()
test_df = pd.DataFrame()
test_df = test_df.append(test['images'], ignore_index=True)
train_df = pd.DataFrame()
train_df = train_df.append(train['images'], ignore_index=True)
train_df_anno = pd.DataFrame()
train_df_anno = train_df_anno.append(train['annotations'], ignore_index=True)
train_df['category_id'] = train_df_anno['category_id'].astype('str')
test_df = test_df.append(test['images'], ignore_index=True)
test_df_anno = pd.DataFrame()

train_df = train_df.drop(['id', 'width', 'height', 'url', 'license'], axis = 1)
test_df = test_df.drop(['id', 'width', 'height', 'url', 'license'], axis = 1)

print(train_df.head())


foldertrain = '/home/garrett/KagglesData/Butterflies/data-images/training/images/'
foldertest = '/home/garrett/KagglesData/Butterflies/data-images/testing/images/'

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=foldertrain,
    x_col="file_name",
    y_col="category_id",
    subset="training",
    batch_size=10,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes = [str(x) for x in range(5419)],
    target_size=(600,600))

valid_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=foldertrain,
    x_col="file_name",
    y_col="category_id",
    subset="validation",
    batch_size=10,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    classes = [str(x) for x in range(5419)],    
    target_size=(600,600))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=test_df,
directory=foldertest,
x_col="file_name",
y_col=None,
batch_size=10,
seed=42,
shuffle=False,
class_mode=None,
target_size=(600,600))


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 294583569467726928
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4327342080
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7198020625127202664
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]
                              file_name category_id
0  0000031e2e9701e24d046c0dc3889bde.jpg         578
1  00001246ce7f88ec292384abcc58d3e0.jpg         438
2  00006fd6c3cd2e525e2df491e157844d.jpg        3990
3  0000a8ef4598a73544df0d42ca754a73.jpg        2964
4  0000d2de78907e422f61ec502819e27b.jpg        1757
Found 355079 validated image filenames belonging to 5419 classes.
Found 118359 validated image filenames belonging to 5419 classes.
Found 118282 validated image filenames.


In [68]:

#the images are all different sizes, so we will need to resize them. Size up to 600x600

#Takes dataframe and image directory, returns 600x600x3x(SIZE) ndarray of images.
#This is very slow, so to avoid looping over the dataframe more times than necessary we combine the function
# to get labels and the function to creat padded images into one.

def getLabelsAndPaddedImages(df, folder, isTrain):
    #start with only a small dataset, to make sure the network architecture is ok
    slicesize = len(df.index)
    #slicesize = 1000
    cat_labels = []
    images = []
    for i in range(slicesize):
        # define subplot
        # define filename
        filename = folder + df.at[i, 'file_name']
        # load image pixels. Images are rgb
        image = imread(filename)
        imresize = cv2.resize(image, (600,600),interpolation = cv2.INTER_AREA)
        #plot raw rgb pixel data
        #if i < 9:
        #    pyplot.subplot(330 + 1 + i)
        #    pyplot.imshow(imresize)
        # show the figure
        #pyplot.show()
        images.append(imresize)
        if(i%100 == 0):
            print("got image "),
            print(i)
        if isTrain == 1:
            labels = np.zeros(5419)
            cat_label = df.at[i,'category_id']
            labels[cat_label] = 1
            cat_labels.append(labels)
    if (isTrain == 1):
            #print(cat_labels)
            return (cat_labels, images)
    else: return images

In [48]:
'''
train_labels_tot,train_img_tot = getLabelsAndPaddedImages(train_df, foldertrain, 1)
test_img = getLabelsAndPaddedImages(test_df, foldertest,0)

#Normalize and standardize images!!!




c = list(zip(train_labels_tot, train_img_tot))
np.random.shuffle(c)
train_labels_tot_shuf, train_img_tot_shuf = zip(*c)

train_labels = np.array(train_labels_tot_shuf[:int(9*len(train_labels_tot_shuf)/10)])
validation_labels = np.array(train_labels_tot_shuf[int(len(train_labels_tot_shuf)/10):])
train_img = np.array(train_img_tot_shuf[:int(9*len(train_labels_tot_shuf)/10)])
validation_img = np.array(train_img_tot_shuf[int(len(train_img_tot_shuf)/10):])

print(len(train_img))
'''

'\ntrain_labels_tot,train_img_tot = getLabelsAndPaddedImages(train_df, foldertrain, 1)\ntest_img = getLabelsAndPaddedImages(test_df, foldertest,0)\n\n#Normalize and standardize images!!!\n\n\n\n\nc = list(zip(train_labels_tot, train_img_tot))\nnp.random.shuffle(c)\ntrain_labels_tot_shuf, train_img_tot_shuf = zip(*c)\n\ntrain_labels = np.array(train_labels_tot_shuf[:int(9*len(train_labels_tot_shuf)/10)])\nvalidation_labels = np.array(train_labels_tot_shuf[int(len(train_labels_tot_shuf)/10):])\ntrain_img = np.array(train_img_tot_shuf[:int(9*len(train_labels_tot_shuf)/10)])\nvalidation_img = np.array(train_img_tot_shuf[int(len(train_img_tot_shuf)/10):])\n\nprint(len(train_img))\n'

In [49]:
#print(train_labels.shape)

NameError: name 'train_labels' is not defined

In [50]:
#Now we build the basic Convnet. Use Tensorflow + Keras for this on my RTX gpu.
#Let's make ResNet from scratch in Keras. First, define the basic residual module.

def identity_block(X, f, filters, stage, block):
    
    # defining name basis
    cNN_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_forlater = X
    
    # First CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = cNN_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('elu')(X)

    # Second CNN, batchnorm, and RELU.
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = cNN_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('elu')(X)

    # Third CNN and batchnorm
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = cNN_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add original X value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_forlater])
    X = Activation('elu')(X)
    
    
    return X

In [51]:
def convolutional_block(X, f, filters, stage, block, s = 2):

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_forlater = X

    # First CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.

    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('elu')(X)

    # Second CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('elu')(X)


    # Third CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    X_forlater = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_forlater)
    X_forlater = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_forlater)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_forlater])
    X = Activation('elu')(X)
    
    return X

In [52]:
def ResNet(input_shape=(600, 600, 3), classes=5419):

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('elu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[4, 4, 16], stage=2, block='a', s=1)
    X = identity_block(X, 3, [4, 4, 16], stage=2, block='b')
    X = identity_block(X, 3, [4, 4, 16], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [8, 8, 32], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [8, 8, 32], stage=3, block='b')
    X = identity_block(X, 3, [8, 8, 32], stage=3, block='c')
    X = identity_block(X, 3, [8, 8, 32], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [16, 16, 64], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='b')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='c')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='d')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='e')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [32,32,128], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [32, 32, 128], stage=5, block='b')
    X = identity_block(X, 3, [32, 32, 128], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [62]:
import tensorflow.keras.backend as K
K.clear_session()

In [69]:
model= ResNet(input_shape=(600, 600, 3))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

/home/garrett/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
  195/35507 [..............................] - ETA: 1:58:42 - loss: 10.4673 - accuracy: 0.0051

In [ ]:
print('\n# Generate predictions for test set')
predictions = model.predict(np.array(test_img))
print('predictions shape:', predictions.shape)
predvals = tf.math.argmax(predictions, axis = 1)
print(predvals)